**Importaciones**

In [2]:
import re
import time
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from pandas.api.types import CategoricalDtype
from sqlalchemy import create_engine
from sqlalchemy import text
from sklearn.linear_model import LogisticRegression, Ridge
from statsmodels.miscmodels.ordinal_model import OrderedModel

# **Extracción**

**Conexión con MySQL**

In [4]:
user      = "root"
password  = "etl25"
host      = "localhost"
port      = "3306"
database  = "icfes_raw"

# Crear conexión SQLAlchemy (driver: mysql-connector)
url = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(url)

**Extracción de datos**

In [5]:
# Leer todos los datos de la tabla
df_mysql = pd.read_sql("SELECT * FROM examen_saber_11", con=engine)
print(df_mysql.shape)
df_mysql.head()


(676508, 84)


,periodo,estu_consecutivo,estu_estudiante,estu_tipodocumento,cole_area_ubicacion,cole_bilingue,cole_calendario,cole_caracter,cole_cod_dane_establecimiento,cole_cod_dane_sede,...,percentil_ingles,percentil_lectura_critica,percentil_matematicas,percentil_sociales_ciudadanas,punt_c_naturales,punt_global,punt_ingles,punt_lectura_critica,punt_matematicas,punt_sociales_ciudadanas
0,20241,SB11202410000447,ESTUDIANTE,TI,URBANO,S,B,ACADÉMICO,311848002351,311848002351,...,89,78,66,91,64,344,84,69,65,72
1,20241,SB11202410095326,ESTUDIANTE,CC,URBANO,None,A,None,311001800618,311001800618,...,23,45,47,24,43,250,47,58,57,43
2,20241,SB11202410095321,ESTUDIANTE,PPT,URBANO,None,B,None,411001800752,411001800752,...,44,23,19,6,53,229,61,49,43,33
3,20241,SB11202410032253,ESTUDIANTE,TI,URBANO,N,B,TÉCNICO/ACADÉMICO,376001007670,376001007670,...,51,28,39,57,63,286,66,51,53,59
4,20241,SB11202410095319,ESTUDIANTE,TI,URBANO,None,B,None,411001800752,411001800752,...,63,46,36,50,55,285,74,59,52,56


**Selección de variables de Interés**

In [6]:
# Variables a excluir del conjunto de datos
excluded_variables = [
    "estu_tipodocumento",
    "estu_consecutivo",
    "cole_cod_dane_establecimiento",
    "cole_cod_dane_sede",
    "cole_cod_depto_ubicacion",
    "cole_cod_mcpio_ubicacion",
    "cole_codigo_icfes",
    "cole_nombre_establecimiento",
    "cole_nombre_sede",
    "cole_sede_principal",
    "estu_agregado",
    "estu_cod_depto_presentacion",
    "estu_cod_mcpio_presentacion",
    "estu_cod_reside_depto",
    "estu_cod_reside_mcpio",
    "estu_discapacidad",
    "estu_etnia",
    "estu_tieneetnia",
    "estu_fechanacimiento",
    "estu_privado_libertad",
    "estu_repite",
    "estu_tieneetnia",
    "estu_tiporemuneracion",
    "fami_situacioneconomica",
    "fami_tieneconsolavideojuegos",
    "fami_tienehornomicroogas",
    "fami_tieneautomovil",
    "fami_tienelavadora",
    "fami_tienemotocicleta",
    "fami_tieneserviciotv",
    "fami_trabajolaborpadre",
    "fami_trabajolabormadre"
]

df_mysql.drop(columns=excluded_variables, inplace=True)

print("Dimensiones después de eliminar:", df_mysql.shape)
df_mysql.head()

Dimensiones después de eliminar: (676508, 53)


,periodo,estu_estudiante,cole_area_ubicacion,cole_bilingue,cole_calendario,cole_caracter,cole_depto_ubicacion,cole_genero,cole_jornada,cole_mcpio_ubicacion,...,percentil_ingles,percentil_lectura_critica,percentil_matematicas,percentil_sociales_ciudadanas,punt_c_naturales,punt_global,punt_ingles,punt_lectura_critica,punt_matematicas,punt_sociales_ciudadanas
0,20241,ESTUDIANTE,URBANO,S,B,ACADÉMICO,BOGOTÁ,FEMENINO,COMPLETA,BOGOTÁ D.C.,...,89,78,66,91,64,344,84,69,65,72
1,20241,ESTUDIANTE,URBANO,None,A,None,BOGOTÁ,MIXTO,SABATINA,BOGOTÁ D.C.,...,23,45,47,24,43,250,47,58,57,43
2,20241,ESTUDIANTE,URBANO,None,B,None,BOGOTÁ,MIXTO,UNICA,BOGOTÁ D.C.,...,44,23,19,6,53,229,61,49,43,33
3,20241,ESTUDIANTE,URBANO,N,B,TÉCNICO/ACADÉMICO,VALLE,MIXTO,TARDE,CALI,...,51,28,39,57,63,286,66,51,53,59
4,20241,ESTUDIANTE,URBANO,None,B,None,BOGOTÁ,MIXTO,UNICA,BOGOTÁ D.C.,...,63,46,36,50,55,285,74,59,52,56


# **Normalización**

In [7]:
df_mysql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676508 entries, 0 to 676507
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   periodo                        676508 non-null  object
 1   estu_estudiante                676508 non-null  object
 2   cole_area_ubicacion            577927 non-null  object
 3   cole_bilingue                  463331 non-null  object
 4   cole_calendario                577927 non-null  object
 5   cole_caracter                  549448 non-null  object
 6   cole_depto_ubicacion           577927 non-null  object
 7   cole_genero                    577927 non-null  object
 8   cole_jornada                   577927 non-null  object
 9   cole_mcpio_ubicacion           577927 non-null  object
 10  cole_naturaleza                577927 non-null  object
 11  desemp_c_naturales             676508 non-null  object
 12  desemp_ingles                  669200 non-nu

Se identificó que todas las variables son de tipo **Object**. Con esto podemos empezar un proceso de **normalización** para que los tipos de datos **correspondan a la naturaleza de las variables**.  

**Variables Categóricas Ordinales**

In [8]:
print(df_mysql['desemp_ingles'].unique())
print(df_mysql['estu_dedicacioninternet'].unique())
print(df_mysql['estu_dedicacionlecturadiaria'].unique())
print(df_mysql['estu_horassemanatrabaja'].unique())
print(df_mysql['fami_comecarnepescadohuevo'].unique())
print(df_mysql['fami_comecerealfrutoslegumbre'].unique())
print(df_mysql['fami_comelechederivados'].unique())
print(df_mysql['fami_cuartoshogar'].unique())
print(df_mysql['fami_estratovivienda'].unique())
print(df_mysql['fami_numlibros'].unique())
print(df_mysql['fami_personashogar'].unique())


print(df_mysql['estu_nse_individual'].unique())
print(df_mysql['estu_nse_establecimiento'].unique())


print(df_mysql['desemp_c_naturales'].unique())
print(df_mysql['desemp_lectura_critica'].unique())
print(df_mysql['desemp_matematicas'].unique())
print(df_mysql['desemp_sociales_ciudadanas'].unique())

['B+' 'A-' 'A2' 'B1' 'A1' None]
['Más de 3 horas' 'Entre 1 y 3 horas' None '30 minutos o menos'
 'Entre 30 y 60 minutos' 'No Navega Internet']
['No leo por entretenimiento' '30 minutos o menos' 'Entre 30 y 60 minutos'
 None 'Entre 1 y 2 horas' 'Más de 2 horas']
['0' 'Más de 30 horas' 'Menos de 10 horas' None 'Entre 11 y 20 horas'
 'Entre 21 y 30 horas']
['Todos o casi todos los días' '3 a 5 veces por semana' None
 'Nunca o rara vez comemos eso' '1 o 2 veces por semana']
['Todos o casi todos los días' '1 o 2 veces por semana'
 '3 a 5 veces por semana' None 'Nunca o rara vez comemos eso']
['Todos o casi todos los días' '3 a 5 veces por semana' None
 '1 o 2 veces por semana' 'Nunca o rara vez comemos eso']
['Tres' 'Cinco' None 'Cuatro' 'Dos' 'Uno' 'Seis o mas']
['Estrato 4' 'Estrato 2' 'Estrato 3' None 'Estrato 5' 'Estrato 6'
 'Estrato 1' 'Sin Estrato']
['MÁS DE 100 LIBROS' '11 A 25 LIBROS' '26 A 100 LIBROS' None
 '0 A 10 LIBROS']
['5 a 6' '3 a 4' None '1 a 2' '7 a 8' '9 o más']
['4' '3' 

**Variables Categóricas Nominales**

In [9]:
print(df_mysql['fami_educacionmadre'].unique())
print(df_mysql['fami_educacionpadre'].unique())

print(df_mysql['estu_estudiante'].unique())
print(df_mysql['cole_calendario'].unique())
print(df_mysql['cole_caracter'].unique())
print(df_mysql['cole_depto_ubicacion'].unique())
print(df_mysql['cole_genero'].unique())
print(df_mysql['cole_jornada'].unique())
print(df_mysql['cole_mcpio_ubicacion'].unique())
print(df_mysql['estu_depto_presentacion'].unique())
print(df_mysql['estu_depto_reside'].unique())
print(df_mysql['estu_mcpio_presentacion'].unique())
print(df_mysql['estu_mcpio_reside'].unique())
print(df_mysql['estu_nacionalidad'].unique())
print(df_mysql['estu_pais_reside'].unique())

print(df_mysql['periodo'].unique())
print(df_mysql['cole_naturaleza'].unique())
print(df_mysql['estu_genero'].unique())
print(df_mysql['cole_area_ubicacion'].unique())

['Postgrado' 'Secundaria (Bachillerato) completa' 'No sabe'
 'Educación profesional completa' None 'Educación profesional incompleta'
 'Primaria completa' 'Técnica o tecnológica completa'
 'Primaria incompleta' 'Secundaria (Bachillerato) incompleta'
 'Técnica o tecnológica incompleta' 'Ninguno' 'No Aplica']
['Postgrado' 'Secundaria (Bachillerato) completa' 'No sabe'
 'Educación profesional completa' 'Técnica o tecnológica completa'
 'Educación profesional incompleta' None
 'Secundaria (Bachillerato) incompleta' 'Primaria completa' 'No Aplica'
 'Primaria incompleta' 'Técnica o tecnológica incompleta' 'Ninguno']
['ESTUDIANTE' 'INDIVIDUAL']
['B' 'A' 'OTRO' None]
['ACADÉMICO' None 'TÉCNICO/ACADÉMICO' 'TÉCNICO' 'NO APLICA']
['BOGOTÁ' 'VALLE' 'QUINDIO' 'CUNDINAMARCA' 'NARIÑO' 'ANTIOQUIA' 'CORDOBA'
 'BOLIVAR' 'CAUCA' 'ATLANTICO' 'SANTANDER' 'RISARALDA' 'CESAR' 'MAGDALENA'
 'HUILA' 'BOYACA' 'CALDAS' 'NORTE SANTANDER' 'META' 'TOLIMA' 'LA GUAJIRA'
 'CASANARE' 'ARAUCA' 'SUCRE' 'CAQUETA' 'PUTUMAYO

**Variables númericas Enteros**

In [10]:
print(df_mysql['estu_grado'].unique())


print(df_mysql['percentil_c_naturales'].unique())
print(df_mysql['percentil_global'].unique())
print(df_mysql['percentil_ingles'].unique())
print(df_mysql['percentil_lectura_critica'].unique())
print(df_mysql['percentil_matematicas'].unique())
print(df_mysql['percentil_sociales_ciudadanas'].unique())

print(df_mysql['punt_c_naturales'].unique())
print(df_mysql['punt_global'].unique())
print(df_mysql['punt_ingles'].unique())
print(df_mysql['punt_lectura_critica'].unique())
print(df_mysql['punt_matematicas'].unique())
print(df_mysql['punt_sociales_ciudadanas'].unique())



['11' '26' '10' None '25' '12']
['71' '22' '42' '68' '48' '99' '43' '93' '26' '74' '17' '100' '32' '85'
 '44' '96' '84' '81' '97' '28' '38' '61' '3' '87' '94' '8' '69' '62' '82'
 '39' '92' '54' '79' '64' '10' '23' '30' '50' '16' '34' '89' '98' '78'
 '19' '15' '31' '36' '59' '80' '66' '40' '63' '65' '4' '29' '76' '21' '6'
 '55' '88' '60' '67' '95' '33' '14' '58' '86' '49' '11' '77' '75' '2' '83'
 '13' '57' '25' '1' '70' '91' '35' '53' '41' '18' '9' '12' '45' '72' '5'
 '52' '7' '51' '27' '90' '56' '46' '24' '47' '20' '37' '73']
['79' '32' '23' '48' '47' '96' '97' '25' '93' '21' '100' '37' '73' '60'
 '84' '81' '99' '24' '44' '78' '2' '74' '9' '62' '88' '43' '46' '90' '38'
 '69' '18' '31' '58' '20' '55' '45' '12' '5' '95' '75' '11' '27' '28' '64'
 '76' '66' '54' '83' '68' '56' '67' '61' '94' None '41' '86' '82' '16' '3'
 '87' '70' '14' '65' '13' '71' '63' '39' '85' '10' '53' '98' '50' '40'
 '91' '8' '36' '52' '42' '34' '26' '33' '35' '89' '4' '72' '1' '15' '6'
 '22' '19' '92' '77' '59' '57

**Variables númericas Decimales**

In [11]:
print(df_mysql['estu_inse_individual'].unique())


['81.453896' '55.711433' '61.920422' ... '41.098202' '59.365536'
 '56.237228']


**Variables Binarias**

In [12]:
print(df_mysql['cole_bilingue'].unique())
print(df_mysql['fami_tienecomputador'].unique())
print(df_mysql['fami_tieneinternet'].unique())

['S' None 'N']
['Si' None 'No']
['Si' None 'No']


**Normalización Variables Categóricas Ordinales**

In [13]:
# helpers
def _norm(s):
    return (pd.Series(s, dtype="string").str.upper().str.strip()
            .str.replace("Á","A").str.replace("É","E").str.replace("Í","I")
            .str.replace("Ó","O").str.replace("Ú","U").str.replace("MÁS","MAS"))

def _cat_ord(col, cats):
    if col in df_mysql.columns:
        df_mysql[col] = _norm(df_mysql[col])
        df_mysql[col] = pd.Categorical(df_mysql[col],
                                    categories=[c.upper() for c in cats],
                                    ordered=True)

# ----- ORDINALES (texto) -----
_cat_ord("desemp_ingles", ["A-","A1","A2","B1","B+"])

_cat_ord("estu_dedicacioninternet", [
    "NO NAVEGA INTERNET","30 MINUTOS O MENOS","ENTRE 30 Y 60 MINUTOS",
    "ENTRE 1 Y 3 HORAS","MAS DE 3 HORAS",
])

_cat_ord("estu_dedicacionlecturadiaria", [
    "NO LEO POR ENTRETENIMIENTO","30 MINUTOS O MENOS","ENTRE 30 Y 60 MINUTOS",
    "ENTRE 1 Y 2 HORAS","MAS DE 2 HORAS",
])

_cat_ord("estu_horassemanatrabaja", [
    "0","MENOS DE 10 HORAS","ENTRE 11 Y 20 HORAS","ENTRE 21 Y 30 HORAS","MAS DE 30 HORAS",
])

for c in ["fami_comecarnepescadohuevo","fami_comecerealfrutoslegumbre","fami_comelechederivados"]:
    _cat_ord(c, [
        "NUNCA O RARA VEZ COMEMOS ESO","1 O 2 VECES POR SEMANA",
        "3 A 5 VECES POR SEMANA","TODOS O CASI TODOS LOS DIAS",
    ])

_cat_ord("fami_numlibros", ["0 A 10 LIBROS","11 A 25 LIBROS","26 A 100 LIBROS","MAS DE 100 LIBROS"])
_cat_ord("fami_personashogar", ["1 A 2","3 A 4","5 A 6","7 A 8","9 O MAS"])

# ----- Cuartos: por número (1..6) + "6 O MAS" -----
if "fami_cuartoshogar" in df_mysql.columns:
    s = _norm(df_mysql["fami_cuartoshogar"]).replace({
        "UNO":"1","DOS":"2","TRES":"3","CUATRO":"4","CINCO":"5","SEIS":"6","SEIS O MAS":"6 O MAS"
    })
    df_mysql["fami_cuartoshogar"] = pd.Categorical(
        s, categories=["1","2","3","4","5","6","6 O MAS"], ordered=True
    )

# ----- Estrato: categórica ordenada con etiquetas de texto -----
# Estrato: categórica ordenada 0..6 (0 = SIN ESTRATO)
s = (_norm(df_mysql["fami_estratovivienda"])
        .str.replace(r"^(ESTRATO\s*)?([1-6])$", r"\2", regex=True)
        .str.replace(r"^SIN\s*ESTRATO$", "0", regex=True))

df_mysql["fami_estratovivienda"] = pd.Categorical(
    s, categories=list("0123456"), ordered=True
)

# ----- Ordinales numéricos -> categoría ordenada -----
for c in ["estu_nse_individual","estu_nse_establecimiento",
            "desemp_c_naturales","desemp_lectura_critica",
            "desemp_matematicas","desemp_sociales_ciudadanas"]:
    if c in df_mysql.columns:
        s = pd.to_numeric(df_mysql[c], errors="coerce")
        df_mysql[c] = pd.Categorical(s, categories=sorted(s.dropna().unique()), ordered=True)


**Normalización Variables númericas Enteros y Decimales**

In [14]:
# ENTEROS: grado, percentiles y puntajes
int_cols = ["estu_grado"] + [c for c in df_mysql.columns if c.startswith("percentil_") or c.startswith("punt_")]
df_mysql[int_cols] = (
    df_mysql[int_cols]
        .apply(pd.to_numeric, errors="coerce")  # convierte texto a número
        .round()                                # por si hay algún decimal perdido
        .astype("Int64")                        # entero con NaN permitido
)

# DECIMAL: INSE individual
if "estu_inse_individual" in df_mysql.columns:
    df_mysql["estu_inse_individual"] = pd.to_numeric(df_mysql["estu_inse_individual"], errors="coerce").astype(float)


**Normalización Variables Binarias**

In [15]:
# Binarias -> 0/1 (Int64), preserva NaN/<NA>
bin_cols = ["cole_bilingue", "fami_tienecomputador", "fami_tieneinternet"]

pos = {"SI","S"}     # valores positivos
neg = {"NO","N"}     # valores negativos
m = {**{v: 1 for v in pos}, **{v: 0 for v in neg}}

for col in bin_cols:
    if col in df_mysql.columns:
        df_mysql[col] = _norm(df_mysql[col]).map(m).astype("Int64")

In [16]:
df_mysql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676508 entries, 0 to 676507
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype   
---  ------                         --------------   -----   
 0   periodo                        676508 non-null  object  
 1   estu_estudiante                676508 non-null  object  
 2   cole_area_ubicacion            577927 non-null  object  
 3   cole_bilingue                  463331 non-null  Int64   
 4   cole_calendario                577927 non-null  object  
 5   cole_caracter                  549448 non-null  object  
 6   cole_depto_ubicacion           577927 non-null  object  
 7   cole_genero                    577927 non-null  object  
 8   cole_jornada                   577927 non-null  object  
 9   cole_mcpio_ubicacion           577927 non-null  object  
 10  cole_naturaleza                577927 non-null  object  
 11  desemp_c_naturales             676508 non-null  category
 12  desemp_ingles   

# **Tratamiento de Datos Nulos**

**Datos Nulos: Cantidad y Porcentaje por Variable**

In [17]:
# Conteo de datos nulos por columna
missing_counts = df_mysql.isnull().sum()
print(" Missing Value Counts per Column:")
display(missing_counts[missing_counts > 0].sort_values(ascending=False))

 Missing Value Counts per Column:


cole_bilingue                    213177
fami_numlibros                   175692
fami_comecerealfrutoslegumbre    163130
estu_dedicacioninternet          162720
fami_comecarnepescadohuevo       162125
estu_dedicacionlecturadiaria     161636
estu_inse_individual             161408
estu_nse_individual              161408
fami_comelechederivados          161304
fami_tieneinternet               160697
fami_educacionmadre              160590
fami_educacionpadre              160261
fami_cuartoshogar                142782
estu_horassemanatrabaja          142574
fami_tienecomputador             142487
fami_personashogar               141459
cole_caracter                    127060
estu_nse_establecimiento          98597
cole_genero                       98581
cole_depto_ubicacion              98581
cole_calendario                   98581
cole_area_ubicacion               98581
cole_mcpio_ubicacion              98581
cole_jornada                      98581
cole_naturaleza                   98581


In [18]:
# % de valores nulos por columna
total_rows = len(df_mysql)
missing_percentage = (df_mysql.isnull().sum() / total_rows) * 100
print("\n Missing Value Percentage per Column:")
display(missing_percentage[missing_percentage > 0].sort_values(ascending=False))


 Missing Value Percentage per Column:


cole_bilingue                    31.511379
fami_numlibros                   25.970425
fami_comecerealfrutoslegumbre    24.113536
estu_dedicacioninternet          24.052931
fami_comecarnepescadohuevo       23.964979
estu_dedicacionlecturadiaria     23.892696
estu_inse_individual             23.858994
estu_nse_individual              23.858994
fami_comelechederivados          23.843620
fami_tieneinternet               23.753895
fami_educacionmadre              23.738078
fami_educacionpadre              23.689446
fami_cuartoshogar                21.105737
estu_horassemanatrabaja          21.074991
fami_tienecomputador             21.062131
fami_personashogar               20.910174
cole_caracter                    18.781744
estu_nse_establecimiento         14.574403
cole_genero                      14.572038
cole_depto_ubicacion             14.572038
cole_calendario                  14.572038
cole_area_ubicacion              14.572038
cole_mcpio_ubicacion             14.572038
cole_jornad

**Imputación/eliminación de registros**

Usamos metodos de regresión y medidas de tendencia central para poder imputar los valores nulos en las variables que tienen más del 10% de valores Nulos. Para las variables tengan menos del 10% en valores nulos, estos registros son eliminados.

Esto se hace para evitar la pérdida de una gran cantidad de información y, sobre todo, prevenir sesgos en los resultados descriptivos.

Predictores usados para imputar: todos los percentil_*, todos los punt_*, estu_grado, y fami_estrato_num (derivado numérico de fami_estratovivienda). Los NA en predictores se rellenan con medianas en X_all sólo para entrenar/predecir (no se escriben al DF).

* Binarias (cole_bilingue, fami_tienecomputador, fami_tieneinternet) → Logística (liblinear, class_weight='balanced').

* Ordinales (lista de las v ordinales) → Ridge sobre códigos 0..K-1 + redondeo y mapeo a la categoría (respeta el orden).

* Continua (estu_inse_individual) → Ridge.

* Nominales (cole_* / fami_educacionmadre/padre) → (Nulo por “DESCONOCIDO”).

* estu_grado → se mantiene y sus NA se imputan por moda.
(estu_grado es numérica discreta con estos valores {10,11,12,25,26}.
No tiene sentido crear un “desconocido” numérico; Asignamos un valor existente.
Usamos moda porque es lo más simple y estable (y coincide con la mediana ≈ 11).)

In [26]:
# ===== COPIA DE TRABAJO =====
df_mysql_2 = df_mysql.copy(deep=True)

# --- normalizador corto (acentos y mayúsculas) ---
def _norm(s):
    return (pd.Series(s, dtype="string").str.upper().str.strip()
            .str.replace("Á","A").str.replace("É","E").str.replace("Í","I")
            .str.replace("Ó","O").str.replace("Ú","U").str.replace("MÁS","MAS"))

# Asegurar dtype CATEGÓRICO ORDENADO en columnas que a veces llegan como object
ord_defs = {
    "fami_comecarnepescadohuevo": [
        "NUNCA O RARA VEZ COMEMOS ESO","1 O 2 VECES POR SEMANA",
        "3 A 5 VECES POR SEMANA","TODOS O CASI TODOS LOS DIAS"
    ],
    "fami_personashogar": ["1 A 2","3 A 4","5 A 6","7 A 8","9 O MAS"],
}
for col, cats in ord_defs.items():
    if col in df_mysql_2.columns:
        df_mysql_2[col] = pd.Categorical(
            _norm(df_mysql_2[col]),
            categories=[c.upper() for c in cats],
            ordered=True
        )

# ===== PREDICTORES BASE (rápidos) =====
df_mysql_2["fami_estrato_num"] = pd.to_numeric(
    df_mysql_2["fami_estratovivienda"].astype("string"), errors="coerce"
)
base = [c for c in df_mysql_2.columns if c.startswith("percentil_") or c.startswith("punt_")]
base += ["estu_grado","fami_estrato_num"]
base = [c for c in base if c in df_mysql_2.columns]

X_all = df_mysql_2[base].astype("float32")
X_all = X_all.fillna(X_all.median(numeric_only=True))
def X(mask): return X_all.loc[mask]

# ===== COLUMNAS A IMPUTAR (>20% nulos) =====
binarias = ["cole_bilingue", "fami_tienecomputador", "fami_tieneinternet"]
ordinales = [
    "fami_numlibros","fami_comecerealfrutoslegumbre","fami_comelechederivados",
    "fami_cuartoshogar","estu_horassemanatrabaja",
    "estu_dedicacioninternet","estu_dedicacionlecturadiaria",
    "estu_nse_establecimiento","estu_nse_individual",
    "fami_comecarnepescadohuevo","fami_personashogar",
]
continua = "estu_inse_individual"

def stratified_idx(y, max_n):
    if len(y) <= max_n: 
        return y.index
    frac = max_n / len(y)
    return (y.to_frame("y")
                .groupby("y", group_keys=False)
                .apply(lambda g: g.sample(frac=frac, random_state=1))
                .index)

# ===== BINARIAS -> LOGÍSTICA =====
MAX_BIN_TRAIN = 80_000
for col in binarias:
    if col not in df_mysql_2: 
        continue
    m_tr = df_mysql_2[col].notna(); m_pr = df_mysql_2[col].isna()
    if m_tr.sum()==0 or m_pr.sum()==0:
        continue
    y = df_mysql_2.loc[m_tr, col].astype(int)
    clf = LogisticRegression(solver="liblinear", class_weight="balanced", max_iter=200)
    idx = stratified_idx(y, MAX_BIN_TRAIN)
    clf.fit(X(m_tr).loc[idx], y.loc[idx])
    p = clf.predict_proba(X(m_pr))[:,1]
    df_mysql_2.loc[m_pr, col] = pd.Series((p>=0.5).astype(np.int8),
                                            index=df_mysql_2.index[m_pr],
                                            dtype="Int64")

# ===== ORDINALES -> RIDGE (0..K-1 y redondeo) =====
MAX_ORD_TRAIN = 60_000
for col in ordinales:
    if col not in df_mysql_2 or not isinstance(df_mysql_2[col].dtype, CategoricalDtype):
        continue
    codes = df_mysql_2[col].cat.codes.replace(-1, np.nan)
    m_tr = codes.notna(); m_pr = codes.isna()
    if m_tr.sum()==0 or m_pr.sum()==0:
        continue
    y = codes.loc[m_tr].astype(float)
    reg = Ridge(alpha=1.0)
    idx = stratified_idx(y.astype(int), MAX_ORD_TRAIN)
    reg.fit(X(m_tr).loc[idx], y.loc[idx])
    pred = np.rint(reg.predict(X(m_pr))).astype(int)
    cats = list(df_mysql_2[col].cat.categories)
    pred = np.clip(pred, 0, len(cats)-1)
    yhat = pd.Categorical([cats[i] for i in pred], categories=cats, ordered=True)
    df_mysql_2.loc[m_pr, col] = yhat

# ===== CONTINUA -> RIDGE =====
if continua in df_mysql_2:
    m_tr = df_mysql_2[continua].notna(); m_pr = df_mysql_2[continua].isna()
    if m_tr.sum()>0 and m_pr.sum()>0:
        reg = Ridge(alpha=1.0)
        reg.fit(X(m_tr), df_mysql_2.loc[m_tr, continua].astype(float))
        df_mysql_2.loc[m_pr, continua] = reg.predict(X(m_pr)).astype(float)

# ===== NOMINALES (cole_* y padres) -> SOLO "DESCONOCIDO" =====
nom_coles = [
    "cole_caracter","cole_area_ubicacion","cole_calendario","cole_depto_ubicacion",
    "cole_genero","cole_jornada","cole_mcpio_ubicacion","cole_naturaleza",
]
parents = ["fami_educacionmadre","fami_educacionpadre"]
all_nominals = nom_coles + parents

for col in all_nominals:
    if col not in df_mysql_2.columns:
        continue

    if col in parents:
        # padres: normalizo y dejo como categórico NOMINAL
        s = _norm(df_mysql_2[col]).astype("string").fillna("DESCONOCIDO")
        df_mysql_2[col] = pd.Categorical(s, ordered=False)
    else:
        # cole_*: solo rellenar NA con "DESCONOCIDO", respetando dtype
        if isinstance(df_mysql_2[col].dtype, CategoricalDtype):
            df_mysql_2[col] = df_mysql_2[col].cat.add_categories(["DESCONOCIDO"]).fillna("DESCONOCIDO")
        else:
            df_mysql_2[col] = df_mysql_2[col].astype("string").fillna("DESCONOCIDO")

# ===== estu_grado -> moda (Int64) =====
if "estu_grado" in df_mysql_2.columns:
    g_mode = df_mysql_2["estu_grado"].mode(dropna=True)
    if not g_mode.empty:
        df_mysql_2["estu_grado"] = (
            df_mysql_2["estu_grado"].fillna(g_mode.iloc[0]).astype("Int64")
        )

# limpiar helper
df_mysql_2.drop(columns=["fami_estrato_num"], errors="ignore", inplace=True)

C:\Users\braya\AppData\Local\Temp\ipykernel_20836\804491017.py:55: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(frac=frac, random_state=1))
C:\Users\braya\AppData\Local\Temp\ipykernel_20836\804491017.py:55: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(frac=frac, random_state=1))
C:\Users\braya\AppData\Local\Temp\ipykernel_20836\804491017.py:55: FutureWarning: Data

In [27]:
# % de valores nulos por columna
total_rows = len(df_mysql_2)
missing_percentage = (df_mysql_2.isnull().sum() / total_rows) * 100
print("\n Missing Value Percentage per Column:")
display(missing_percentage[missing_percentage > 0].sort_values(ascending=False))


 Missing Value Percentage per Column:


fami_estratovivienda       5.841320
desemp_ingles              1.080253
punt_ingles                1.080253
percentil_global           1.080253
percentil_ingles           1.080253
estu_mcpio_reside          0.656903
estu_depto_reside          0.656903
estu_depto_presentacion    0.002809
estu_mcpio_presentacion    0.002809
estu_genero                0.000443
estu_pais_reside           0.000148
estu_nacionalidad          0.000148
dtype: float64

In [28]:
df_mysql_2.head(25)

,periodo,estu_estudiante,cole_area_ubicacion,cole_bilingue,cole_calendario,cole_caracter,cole_depto_ubicacion,cole_genero,cole_jornada,cole_mcpio_ubicacion,...,percentil_ingles,percentil_lectura_critica,percentil_matematicas,percentil_sociales_ciudadanas,punt_c_naturales,punt_global,punt_ingles,punt_lectura_critica,punt_matematicas,punt_sociales_ciudadanas
0,20241,ESTUDIANTE,URBANO,1,B,ACADÉMICO,BOGOTÁ,FEMENINO,COMPLETA,BOGOTÁ D.C.,...,89,78,66,91,64,344,84,69,65,72
1,20241,ESTUDIANTE,URBANO,1,A,DESCONOCIDO,BOGOTÁ,MIXTO,SABATINA,BOGOTÁ D.C.,...,23,45,47,24,43,250,47,58,57,43
2,20241,ESTUDIANTE,URBANO,1,B,DESCONOCIDO,BOGOTÁ,MIXTO,UNICA,BOGOTÁ D.C.,...,44,23,19,6,53,229,61,49,43,33
3,20241,ESTUDIANTE,URBANO,0,B,TÉCNICO/ACADÉMICO,VALLE,MIXTO,TARDE,CALI,...,51,28,39,57,63,286,66,51,53,59
4,20241,ESTUDIANTE,URBANO,1,B,DESCONOCIDO,BOGOTÁ,MIXTO,UNICA,BOGOTÁ D.C.,...,63,46,36,50,55,285,74,59,52,56
5,20241,ESTUDIANTE,URBANO,0,B,ACADÉMICO,QUINDIO,MIXTO,COMPLETA,ARMENIA,...,64,100,88,93,78,382,75,81,74,73
6,20241,ESTUDIANTE,URBANO,0,B,ACADÉMICO,VALLE,MIXTO,COMPLETA,CALI,...,40,49,60,39,53,284,58,60,63,51
7,20241,ESTUDIANTE,RURAL,0,B,ACADÉMICO,CUNDINAMARCA,MIXTO,COMPLETA,CAJICÁ,...,94,97,95,96,72,384,87,77,79,76
8,20241,ESTUDIANTE,URBANO,1,OTRO,DESCONOCIDO,BOGOTÁ,MIXTO,SABATINA,BOGOTÁ D.C.,...,24,29,20,31,45,234,48,52,43,47
9,20241,ESTUDIANTE,URBANO,1,OTRO,DESCONOCIDO,BOGOTÁ,MIXTO,SABATINA,BOGOTÁ D.C.,...,100,92,86,96,65,371,100,74,73,76


**Eliminación de nulos**

In [29]:
# 1.) Tamaño antes
print("Antes:", df_mysql_2.shape)

# 2) Eliminar filas con cualquier NaN (sin tocar df_mysql2)
data_transformed = df_mysql_2.dropna().copy()

# 3) Reporte de pérdida vs. df_mysql2
print("Después (dropna global):", data_transformed.shape)
perdidas = len(df_mysql_2) - len(data_transformed)
pct = perdidas / len(df_mysql_2) * 100


print(f"Filas eliminadas: {perdidas} ({pct:.2f}%)")

Antes: (676508, 53)
Después (dropna global): (626339, 53)
Filas eliminadas: 50169 (7.42%)


In [ ]:
Path("../data_transformed").mkdir(parents=True, exist_ok=True)
data_transformed.to_csv("../data_transformed/data_icfes_2024.csv", index=False)
#data_transformed.to_excel("../data_transformed/data_icfes_2024.xlsx", index=False)

# **Estadisticas de limpieza**

In [42]:
# Columnas 
cols_ini  = 84
cols_fin  = df_mysql_2.shape[1]
cols_elim = cols_ini - cols_fin
pct_cols  = (cols_elim / cols_ini) * 100

# Filas 
filas_ini  = df_mysql_2.shape[0]
filas_fin  = data_transformed.shape[0]
filas_elim = filas_ini - filas_fin
pct_filas  = (filas_elim / filas_ini) * 100

# Resumen
print("📊 Resumen de limpieza")
print(f"- Columnas: {cols_ini} originales → {cols_elim} eliminadas ({pct_cols:.2f}%) → {cols_fin} finales")
print(f"- Filas: {filas_ini:,} originales → {filas_elim:,} eliminadas ({pct_filas:.2f}%) → {filas_fin:,} finales")


📊 Resumen de limpieza
- Columnas: 84 originales → 31 eliminadas (36.90%) → 53 finales
- Filas: 676,508 originales → 50,169 eliminadas (7.42%) → 626,339 finales


# **Inserción**

**Conexión con MySQL y creación de new DATABASE**

In [33]:
with engine.connect() as conn:
    conn.execute(text("CREATE DATABASE IF NOT EXISTS icfes_transformed;"))


new_db = "icfes_transformed"
url_new = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{new_db}"
engine_new = create_engine(url_new)

**Inserción de datos**

In [34]:

data_transformed.to_sql(
    "data_transformed",
    con=engine_new,
    if_exists="replace",
    index=False,
    chunksize=10000
)


rows, cols = data_transformed.shape
print(f"✅ Data insertada en {new_db}.data_transformed")
print(f"📊 Filas insertadas: {rows:,} | Columnas insertadas: {cols}")


✅ Data insertada en icfes_transformed.data_transformed
📊 Filas insertadas: 626,339 | Columnas insertadas: 53
